## Computer Simulation Project Documentation
### Full Names : MohammadDanial Jahanbani, Erfan Sadrayie
Student Number : 99105367, 99101835
___
## Libraries
We used `Simpy` to simulate the 1-core system as a resource in our code and finally used `Pandas` to display the data for tasks.
## Implementation
based on the system description we implemented the below algorithm. It consists of three main parts:
</br>
</br>
<li>
    <b>Creating jobs</b>: with time intervals $t_i$ exponentially distributed with rate $X$, jobs are created with a service time also exponentially distributed with rate $Y$ plus an exponentially distributed renege time with parameter $Z$. In addition, a priority is generated for this job which equals:$$pq\_priority = creation\_time \times c + priority$$ where $c$ indicates the imporance of creation time in proportion to the actual priority i.e 1 for high, 2 for normal and 3 for low.
</li>
</br>
<li>
    <b>Transferring jobs from layer 1 to Round Robin 1 </b>: In an iterative fashion, after each $T$ seconds the total number of tasks in second layer queues is checked and if it is less than $k$, the first $k$ elements are added to <i>Round Robin 1</i> priority queue.
</li>
</br>
<li>
    <b>Transferring jobs from a queue in layer 2 to the CPU core</b>: Based on our queue selection policy, we can operate two different queue selection for the core. if the policy is <i>Normal</i> then  at First the core will perform tasks from the <i>Round Robin 1</i> priority queue and if that is empty then it will accept tasks from <i>Round Robin 2</i> Priority queue and finally <i>FCFS</i> priority queue. However, if the policy is <i>Random</i> then the core will be provided with a task randomly selected from one of the three queues in layer 2.
</li>
</br>
<li>
    <b>Transferring jobs between queues of layer 2</b>: If a job in layer 2 has a service time higher than the quantum time of the queue it is currently in, then it should be moved to a queue with lower priority after it has been given service by the core equal to the quantum time of the queue it is currently in. 
</li>
</br>
<li>
    <b>Checking for reneged jobs</b>: We iteratively check to see if any jobs have timed out and remove them from the queue they are currently in. If a job is being served by the core, it will not be affected by this. 
</li>

In [1]:
from main import simulate
from numpy import mean

results = []
for x in range(5, 15, 2):
    for y in range(5, 15, 2):
        for z in range(0, 1000, 100):
            jobs_result, queue_result = simulate(x, y, z)
            print('entrance rate:', x, 'service rate:', y)
            print('average PQ length:', mean(queue_result['PQ length']))
            print('average R1 length:', mean(queue_result['R1 length']))
            print('average R2 length', mean(queue_result['R2 length']))
            print('average FCFS length', mean(queue_result['FCFS length']))

entrance rate: 5 service rate: 5
average PQ length: 0.00575
average R1 length: 0.0
average R2 length 0.0
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 0.72225
average R1 length: 1.43625
average R2 length 0.0
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 1.42425
average R1 length: 0.3835
average R2 length 0.05025
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 2.80725
average R1 length: 0.98875
average R2 length 0.15425
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 3.62975
average R1 length: 0.957
average R2 length 0.02775
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 3.395
average R1 length: 1.05725
average R2 length 0.094
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 5.17275
average R1 length: 1.19475
average R2 length 0.04875
average FCFS length 0.00025
entrance rate: 5 service rate: 5
average PQ length: 5.20

entrance rate: 7 service rate: 7
average PQ length: 1.44375
average R1 length: 0.327
average R2 length 0.0205
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 1.53425
average R1 length: 0.707
average R2 length 0.02675
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 3.2165
average R1 length: 1.54375
average R2 length 1.301
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 4.72775
average R1 length: 1.59075
average R2 length 0.21025
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 5.06175
average R1 length: 7.037
average R2 length 0.708
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 5.2825
average R1 length: 2.37825
average R2 length 0.56275
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 5.187
average R1 length: 1.50525
average R2 length 0.14925
average FCFS length 0.0
entrance rate: 7 service rate: 7
average PQ length: 7.30

entrance rate: 9 service rate: 9
average PQ length: 2.13375
average R1 length: 1.12775
average R2 length 0.358
average FCFS length 0.0
entrance rate: 9 service rate: 9
average PQ length: 3.10475
average R1 length: 1.04225
average R2 length 0.055
average FCFS length 0.0
entrance rate: 9 service rate: 9
average PQ length: 4.241
average R1 length: 1.33525
average R2 length 0.15525
average FCFS length 0.0
entrance rate: 9 service rate: 9
average PQ length: 3.2855
average R1 length: 1.2735
average R2 length 0.1425
average FCFS length 0.0
entrance rate: 9 service rate: 9
average PQ length: 4.24325
average R1 length: 2.24575
average R2 length 0.159
average FCFS length 0.0
entrance rate: 9 service rate: 9
average PQ length: 7.7115
average R1 length: 4.08075
average R2 length 0.97225
average FCFS length 0.0
entrance rate: 9 service rate: 9
average PQ length: 5.22
average R1 length: 4.97
average R2 length 0.21975
average FCFS length 0.0
entrance rate: 9 service rate: 11
average PQ length: 0.007


entrance rate: 11 service rate: 11
average PQ length: 3.12525
average R1 length: 4.93725
average R2 length 1.2015
average FCFS length 0.00025
entrance rate: 11 service rate: 11
average PQ length: 2.19825
average R1 length: 2.63325
average R2 length 0.111
average FCFS length 0.0
entrance rate: 11 service rate: 11
average PQ length: 2.83475
average R1 length: 1.01
average R2 length 0.41875
average FCFS length 0.00025
entrance rate: 11 service rate: 11
average PQ length: 5.6055
average R1 length: 1.4595
average R2 length 1.51525
average FCFS length 0.0
entrance rate: 11 service rate: 11
average PQ length: 5.55425
average R1 length: 1.99125
average R2 length 1.03125
average FCFS length 0.0
entrance rate: 11 service rate: 11
average PQ length: 4.95475
average R1 length: 1.648
average R2 length 1.2585
average FCFS length 0.00025
entrance rate: 11 service rate: 13
average PQ length: 0.007
average R1 length: 0.0
average R2 length 0.0
average FCFS length 0.0
entrance rate: 11 service rate: 13
a

entrance rate: 13 service rate: 13
average PQ length: 3.9985
average R1 length: 4.48525
average R2 length 1.38
average FCFS length 0.0
entrance rate: 13 service rate: 13
average PQ length: 4.73475
average R1 length: 4.0085
average R2 length 0.49125
average FCFS length 0.0
entrance rate: 13 service rate: 13
average PQ length: 5.8435
average R1 length: 18.47625
average R2 length 2.09525
average FCFS length 0.00025
entrance rate: 13 service rate: 13
average PQ length: 6.15925
average R1 length: 9.06825
average R2 length 1.40525
average FCFS length 0.0
entrance rate: 13 service rate: 13
average PQ length: 6.9715
average R1 length: 5.23775
average R2 length 0.96975
average FCFS length 0.0


In [4]:
x = 20
y = 30
z = 200
jobs_result, queue_result = simulate(x, y, z)
print(jobs_result)

    id  priority  service time  created time  is done  PQ wt  R1 wt  R2 wt  \
0    1         3             3             0     True  420.0  390.9    6.7   
1    2         1             4            10     True   80.0   90.7   10.0   
2    3         3             5            12    False    NaN    NaN    NaN   
3    4         3            26            38    False   82.0   99.7   11.7   
4    5         3             0            51     True   39.0  104.7    NaN   
5    6         3             5            92     True  418.0  517.6   45.1   
6    7         3            19           124    False    NaN    NaN    NaN   
7    8         3             6           190    False  110.0  315.7    NaN   
8    9         3            45           226    False   44.0  258.1    NaN   
9   10         3             4           230     True  130.0  386.9    NaN   
10  11         1             3           239     True  361.0  601.7   20.3   
11  12         3             4           272     True  148.0  41